In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq


#os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")


llm=ChatGroq(model="qwen-2.5-32b")
#llm = ChatOpenAI(model="gpt-4o")
result=llm.invoke("Hello")
result

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 30, 'total_tokens': 40, 'completion_time': 0.05, 'prompt_time': 0.003685376, 'queue_time': 0.053871884, 'total_time': 0.053685376}, 'model_name': 'qwen-2.5-32b', 'system_fingerprint': 'fp_c527211fd1', 'finish_reason': 'stop', 'logprobs': None}, id='run-b8f2fcdb-b5b7-44f9-bd7d-85b16ffe8ee9-0', usage_metadata={'input_tokens': 30, 'output_tokens': 10, 'total_tokens': 40})

In [ ]:
### State
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

class State(TypedDict):
    user_input: str
    user_input_requirements: str
    user_stories: str
    po_review: str
    design_documents: str
    revised_user_stories: str
    code: str
    
    pass

In [ ]:
### Nodes Definition

def UserInputRequirements(state: State):
    """LLM Call to get the requirements for the user input"""
        # Use the LLM to generate requirements based on the user input
    prompt = f"Write a clear and concise description of the requirements for the following user input: {state['user_input']}"
    response = llm.invoke(prompt)
     # Extract the generated requirements from the LLM response
    requirements = response.content
    return {"user_input_requirements": requirements}

def AutoGenerateUserStories(state: State):
    """LLM Call to auto-generate user stories"""
    # Use the LLM to generate user stories based on the user input and requirements
    prompt = f"Write user stories in the format 'As a [user], I want to [perform some task] so that [achieve some goal]' for the following user input and requirements: {state['user_input']}. Requirements: {state['user_input_requirements']}"
    response = llm.invoke(prompt)
    # Extract the generated user stories from the LLM response
    user_stories = response.content
    return {"user_stories": user_stories}

def ProdutOwnerReview(state: State):
    """Product Owner reviews the user stories"""
    # Product Owner reviews the user stories and provides feedback
    # user_stories = state["user_stories"]
    prompt = f"Provide suggestions to improve the clarity and conciseness of the following user stories: {state['user_stories']} accordingly it will be Approved or it needs some changes i.e Feedback"
    response = llm.invoke(prompt)
    feedback = response.content
    # feedback = "The user stories are clear and concise. I approve them."
    return {"po_review": feedback}

def Check_ProdutOwnerReview(state: State):
    """Check if the Product Owner has approved the user stories"""
    po_review = state["po_review"]
    # Check if the Product Owner has approved the user stories
    # Use a case-insensitive check for keywords indicating approval
    approval_keywords = ["approve", "approved", "accept", "accepted"]
    if any(keyword in po_review.lower() for keyword in approval_keywords):
        return "Approved"
    else:
        return "Feedback"

def DesignDocuments(state: State):
    """Create Design Docuemnents - Functional and technical"""
    # Create design documents based on the user stories
    # user_stories = state["user_stories"]
    prompt = f"Create detailed functional and technical design documents for the following approved user stories: {state['user_stories']}. Ensure the documents include system architecture, data flow, and key technical specifications. Make sure that user stories to consider here should be approved by the Product Owner."
    response = llm.invoke(prompt)
    design_documents = response.content
    return {"design_documents": design_documents}

def ReviseUserStories(state: State):
    """Revise user stories based on the Feedback from the Product Owner Review"""
    feedback = state["po_review"]
    # user_stories = state["user_stories"]
    prompt = f"Revise the following user stories based on the feedback provided by the Product Owner: {state['user_stories']}. Feedback: {feedback}"
    response = llm.invoke(prompt)
    revised_user_stories = response.content
    return {"revised_user_stories": revised_user_stories}

def DesignReview(state: State):
    """Review of the Design Documents by the Product Owner"""
    design_documents = state["design_documents"]
    prompt = f"Review the following design documents : {design_documents} and provide feedback whether it meets the requirements accordingly it will be Approved or it needs some changes i.e Feedback"
    response = llm.invoke(prompt)
    feedback = response.content
    return {"design_review": feedback}


def Check_DesignReview(state: State):
    """Do the check of the Design Review"""
    design_review = state["design_review"]
    # Check if the Product Owner has approved the design documents
    # Use a case-insensitive check for keywords indicating approval
    approval_keywords = ["approve", "approved", "accept", "accepted"]
    if any(keyword in design_review.lower() for keyword in approval_keywords):
        return "Approved"
    else:
        return "Feedback"
    
def GenerateCode(state: State):
    """Generate Code based on the Design Documents"""
    # Generate code based on the design documents
    # design_documents = state["design_review"]
    prompt = f"Generate code based on the following approved design documents: {state['design_documents']} and {state['design_review']}. Ensure the code is clean, well-documented, and follows best practices."
    response = llm.invoke(prompt)
    code = response.content
    return {"code": code}


In [ ]:
# Build Worklow

workflow = StateGraph()

# Add Nodes
workflow.add_node("UserInputRequirements", UserInputRequirements)
workflow.add_node("AutoGenerateUserStories", AutoGenerateUserStories)
# workflow.add_node("ProdonutOwnerReview", Check_ProdutOwnerReview, {"Approved":"DesignDocuments", "Feedback":"ReviseUserStories"})
workflow.add_node("ProdutOwnerReview", ProdutOwnerReview)
workflow.add_node("DesignDocuments", DesignDocuments)
workflow.add_node("ReviseUserStories", ReviseUserStories)
workflow.add_node("DesignReview", DesignReview)
workflow.add_node("GenerateCode", GenerateCode)
workflow.add_node("CodeReview", CodeReview)
workflow.add_node("SecurityReview", SecurityReview)
workflow.add_node("FixCode", FixCode)
workflow.add_node("WriteTestCases", WriteTestCases)
workflow.add_node("FixCodeSecurity", FixCodeSecurity)
workflow.add_node("TestCasesReview", TestCasesReview)
workflow.add_node("QATesting", QATesting)
workflow.add_node("FixTestCases", FixTestCases)
workflow.add_node("FixCode", FixCode)
workflow.add_node("Deploy", Deploy)
workflow.add_node("MonitorFeedback", MonitorFeedback)
workflow.add_node("MaintenanceUpdate", MaintenanceUpdate)


# Add Edges
workflow.add_edge(START, "UserInputRequirements")
workflow.add_edge("UserInputRequirements", "AutoGenerateUserStories")
workflow.add_edge("AutoGenerateUserStories", "ProdutOwnerReview")
workflow.add_conditional_edges("ProdutOwnerReview", Check_ProdutOwnerReview, {"Approved":"DesignDocuments", "Feedback":"ReviseUserStories"})
workflow.add_edge("ReviseUserStories", "AutoGenerateUserStories")
workflow.add_edge("DesignDocuments", "DesignReview")
workflow.add_conditional_edges("DesignReview", Check_DesignReview, {"Approved":"GenerateCode", "Feedback":"DesignDocuments"})
workflow.add_edge("GenerateCode", "CodeReview")
workflow.add_conditional_edges("CodeReview", Check_CodeReview, {"Approved":"SecurityReview", "Feedback":"FixCode"})
workflow.add_edge("FixCode", "GenerateCode")
workflow.add_conditional_edges("SecurityReview", Check_SecurityReview, {"Approved":"WriteTestCases", "Feedback":"FixCodeSecurity"})
workflow.add_edge("FixCodeSecurity", "GenerateCode")
workflow.add_edge("WriteTestCases", "TestCasesReview")
workflow.add_conditional_edges("TestCasesReview", Check_TestCasesReview, {"Approved":"QATesting", "Feedback":"FixTestCases"})
workflow.add_edge("FixTestCases", "WriteTestCases")
workflow.add_conditional_edges("QATesting", Check_QATesting, {"Passed":"Deploy", "Failed":"FixCode"})
workflow.add_edge("FixCode", "GenerateCode")
workflow.add_edge("Deploy", "MonitorFeedback")
workflow.add_edge("MonitorFeedback", "MaintenanceUpdate")
workflow.add_edge("MaintenanceUpdate", END)

# Compile Workflow
graph = workflow.compile()

# Show workflow
display(Image(graph.get_graph().draw_mermaid_png()))